In [9]:
import pandas as pd
import numpy as np
from math import sqrt, log, exp

from sklearn.preprocessing import MinMaxScaler

In [10]:
# read files
tn_data = pd.read_csv('../dat/train.csv', index_col='Id')
tt_data = pd.read_csv('../dat/test.csv', index_col='Id')

# split x & y
tn_y = tn_data['SalePrice']
tn_x = tn_data.drop(['SalePrice'], axis=1)
tt_x = tt_data

# filter out NA columns
tn_null = tn_x.isnull().any()
tt_null = tt_x.isnull().any()
na_filter = ~(tn_null | tt_null)
tn_x = tn_x.loc[:, na_filter]
tt_x = tt_x.loc[:, na_filter]
#tn_x = tn_x.fillna(0)
#tt_x = tt_x.fillna(0)

# split numerical & categorical features
ncol = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 
        'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 
        'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 
        'MiscVal', 'MoSold', 'YrSold']
tn_nx = tn_x[ncol]
tn_cx = tn_x.drop(ncol, axis=1)
tt_nx = tt_x[ncol]
tt_cx = tt_x.drop(ncol, axis=1)

# join numerical/categorical tn & tt
cx = pd.concat([tn_cx, tt_cx], axis=0)
nx = pd.concat([tn_nx, tt_nx], axis=0)

# scale numerical features
scaler = MinMaxScaler()
nx = pd.DataFrame(scaler.fit_transform(nx), index=nx.index.values, columns=nx.columns.values)

# split numerical tn & tt
tn_nx = nx.iloc[:tn_nx.shape[0],]
tt_nx = nx.iloc[tn_nx.shape[0]:,]

# MSSubClass as a categorical feature
cx['MSSubClass'] = cx['MSSubClass'].astype('object')

# one-hot-encoding categorical features
cx = pd.get_dummies(cx)

# split categorical tn & tt
tn_cx = cx.iloc[:tn_nx.shape[0],]
tt_cx = cx.iloc[tn_nx.shape[0]:,]

# join numerical & categorical features
tn_x = pd.concat([tn_nx, tn_cx], axis=1)
tt_x = pd.concat([tt_nx, tt_cx], axis=1)

# logarithm of tn_y
tn_ly = tn_y.apply(log)

In [11]:
from sklearn.model_selection import ShuffleSplit
# split stn & stt
rs = ShuffleSplit(n_splits=1, test_size=.5, random_state=514)
stn_idx, stt_idx = next(rs.split(tn_x),None)
stn_x = tn_x.iloc[stn_idx,]
stt_x = tn_x.iloc[stt_idx,]
stn_ly = tn_ly.iloc[stn_idx,]
stt_ly = tn_ly.iloc[stt_idx,]

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR, SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

model_lst = []

# LinearRegression
model_lst.append(['LinearRegression',
                  [['normalize=%r' % (n), LinearRegression(normalize=n)] for n in [False, True]]])
    
# RandomForestRegressor
model_lst.append(['RandomForestRegressor',
                  [['max_depth=%d, n_estimators=%d' % (md, ne), 
                    RandomForestRegressor(max_depth=md, n_estimators=ne, random_state=514)] 
                   for md in [5, 10, 30] 
                   for ne in [1000]]])

# GradientBoostingRegressor
model_lst.append(['GradientBoostingRegressor', 
                  [['max_depth=%d, n_estimators=%d' % (md, ne), 
                    GradientBoostingRegressor(max_depth=md, n_estimators=ne, random_state=514)]
                   for md in [1, 3, 5] 
                   for ne in [1000]]])

# MLPRegressor
model_lst.append(['MLPRegressor_LBFGS',
                  [['learning_rate_init=%f, max_iter=%d' % (0.03, 10000),
                    MLPRegressor(hidden_layer_sizes=(1,), activation='logistic', alpha=0.5, 
                                 random_state=514, solver='lbfgs', verbose=True)]]])

# SVR-RBF
model_lst.append(['SVR-RBF',
                  [['C=%f, eps=%f, gamma=%f' % (c, e, g), 
                    SVR(kernel='rbf', C=c, epsilon=e, gamma=g, cache_size=200)] 
                   for c in [1e0, 1e1, 1e2, 1e3]
                   for e in [0.0, 0.1, 0.2]
                   for g in [1e-3, 1e-2, 1e-1]]])

# SVR-Linear           
model_lst.append(['SVR-Linear',
                  [['C=%f, eps=%f, gamma=%f' % (c, e, g), 
                    SVR(kernel='linear', C=c, epsilon=e, gamma=g, cache_size=200)] 
                   for c in [1e0, 1e1]
                   for e in [0.0, 0.1, 0.2] 
                   for g in [1e-3, 1e-2, 1e-1]]])

# SVR-Poly3
model_lst.append(['SVR-Poly3',
                  [['C=%f, eps=%f, gamma=%f' % (c, e, g), 
                    SVR(kernel='poly', degree=3, C=c, epsilon=e, gamma=g, cache_size=200)] 
                   for c in [1e0, 1e1]
                   for e in [0.0, 0.1, 0.2]
                   for g in [1e-3, 1e-2, 1e-1, 1e0]]])             

verbose=1
for name, models in model_lst:
    print '[%s]' % name
                   
    mmin = float('inf')

    for param, model in models:
        model.fit(stn_x, stn_ly)   

        stn_lp = model.predict(stn_x)
        stt_lp = model.predict(stt_x)

        stn_rmse = sqrt(mean_squared_error(stn_ly, stn_lp))
        stt_rmse = sqrt(mean_squared_error(stt_ly, stt_lp))
                   
        if stt_rmse < mmin:
            mmin = stt_rmse
            b_param = param
            b_stn_rmse = stn_rmse
            b_stt_rmse = stt_rmse

        if verbose >= 2:
            print '%s: stn(%f), stt(%f)' % (param, stn_rmse, stt_rmse)
                   
    if verbose >= 1:
        print '[best]\t %s: stn(%f), stt(%f)' % (b_param, b_stn_rmse, b_stt_rmse)
    print ''

[LinearRegression]
[best]	 normalize=False: stn(0.097538), stt(1059730924.595619)

[RandomForestRegressor]
[best]	 max_depth=30, n_estimators=1000: stn(0.050464), stt(0.178868)

[GradientBoostingRegressor]
[best]	 max_depth=1, n_estimators=1000: stn(0.093590), stt(0.165821)

[MLPRegressor_LBFGS]
[best]	 learning_rate_init=0.030000, max_iter=10000: stn(0.108247), stt(0.160601)

[SVR-RBF]
[best]	 C=10.000000, eps=0.100000, gamma=0.010000: stn(0.090390), stt(0.166733)

[SVR-Linear]
[best]	 C=1.000000, eps=0.100000, gamma=0.001000: stn(0.102425), stt(0.174340)

[SVR-Poly3]
[best]	 C=1.000000, eps=0.100000, gamma=0.100000: stn(0.076831), stt(0.174104)



In [15]:
b_models = [['LinearRegression', 
             LinearRegression(normalize=False)],
            ['RandomForestRegressor', 
             RandomForestRegressor(max_depth=30, n_estimators=1000, random_state=514)], 
            ['GradientBoostingRegressor', 
             GradientBoostingRegressor(max_depth=1, n_estimators=1000, random_state=514)], 
            ['MLPRegressor', 
             MLPRegressor(hidden_layer_sizes=(5,), learning_rate_init=0.03, max_iter=10000, random_state=514)], 
            ['MLPRegressor_LBFGS', 
             MLPRegressor(hidden_layer_sizes=(1,), activation='logistic', alpha=0.5, 
                          random_state=514, solver='lbfgs')],
            ['MLPRegressor', 
             MLPRegressor(hidden_layer_sizes=(1,), activation='logistic', alpha=0.5, random_state=514, solver='lbfgs')], 
            ['SVR-RBF', 
             SVR(kernel='rbf', C=10, epsilon=0.1, gamma=0.01, cache_size=200)], 
            ['SVR-Linear', 
             SVR(kernel='linear', C=1, epsilon=0.1, gamma=0.001, cache_size=200)], 
            ['SVR-Poly3', 
             SVR(kernel='poly', degree=3, C=1, epsilon=0.1, gamma=0.1, cache_size=200)]]

for name, model in b_models:
    model.fit(tn_x, tn_ly)
    tn_lp = model.predict(tn_x)
    tn_rmse = sqrt(mean_squared_error(tn_ly, tn_lp))
    print tn_rmse
    
    tt_lp = model.predict(tt_x)
    tt_p = np.exp(tt_lp)
    idx = tt_x.index.values
    
    '''
    with open('../res/%s.txt' % name, 'w') as f:
        f.write('Id,SalePrice\n')
        for i, p in zip(idx, tt_p):
            f.write(','.join(map(str, [i, p])) + '\n')
    '''

0.115975241928
0.0548842578747
0.10963727231
0.138488109815
0.125251842487
0.125251842487
0.103368022843
0.123006857126
0.0796455747166
